# Chopped NLP
Data courtesy of Jeffrey Braun (https://www.kaggle.com/jeffreybraun/chopped-10-years-of-episode-data)

### Import necessary modules

In [131]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

### Read CSV of Chopped Episode data

In [132]:
chopped_data = pd.read_csv("chopped.csv")

In [159]:
chopped_data.head()

,season,season_episode,series_episode,episode_name,episode_notes,air_date,judge1,judge2,judge3,appetizer,entree,dessert,contestant1,contestant1_info,contestant2,contestant2_info,contestant3,contestant3_info,contestant4,contestant4_info
0,1,1,1,"""Octopus, Duck, Animal Crackers""",This is the first episode with only three offi...,13-Jan-09,Marc Murphy,Alex Guarnaschelli,Aarón Sánchez,"baby octopus, bok choy, oyster sauce, smoked ...","duck breast, green onions, ginger, honey","prunes, animal crackers, cream cheese",Summer Kriegshauser,Private Chef and Nutrition Coach New York NY,Perry Pollaci,Private Chef and Sous chef Bar Blanc New Yo...,Katie Rosenhouse,Pastry Chef Olana Restaurant New York NY,Sandy Davis,Catering Chef Showstoppers Catering at Union...
1,1,2,2,"""Tofu, Blueberries, Oysters""",This is the first of a few episodes with five ...,20-Jan-09,Aarón Sánchez,Alex Guarnaschelli,Marc Murphy,"firm tofu, tomato paste, prosciutto","daikon, pork loin, Napa cabbage, Thai chiles,...","phyllo dough, gorgonzola cheese, pineapple ri...",Raymond Jackson,Private Caterer and Culinary Instructor West...,Klaus Kronsteiner,Chef de cuisine Liberty National Golf Course...,Christopher Jackson,Executive Chef and Owner Ted and Honey Broo...,Pippa Calland,Owner and Chef Chef for Hire LLC Newville PA
2,1,3,3,"""Avocado, Tahini, Bran Flakes""",NaN,27-Jan-09,Aarón Sánchez,Alex Guarnaschelli,Marc Murphy,"lump crab meat, dried shiitake mushrooms, pin...","ground beef, cannellini beans, tahini paste, ...","brioche, cantaloupe, pecans, avocados",Margaritte Malfy,Executive Chef and Co-owner La Palapa New Y...,Rachelle Rodwell,Chef de cuisine SoHo Grand Hotel New York NY,Chris Burke,Private Chef New York NY,Andre Marrero,Chef tournant L’Atelier de Joël Robuchon Ne...
3,1,4,4,"""Banana, Collard Greens, Grits""","In the appetizer round, Chef Chuboda refused t...",3-Feb-09,Scott Conant,Amanda Freitag,Geoffrey Zakarian,"ground beef, wonton wrappers, cream of mushro...","scallops, collard greens, anchovies, sour cream","maple syrup, black plums, almond butter, waln...",Sean Chudoba,Executive Chef Ayza Wine Bar New York NY,Kyle Shadix,Chef Registered Dietician and Culinary Consu...,Luis Gonzales,Executive Chef Knickerbocker Bar & Grill Ne...,Einat Admony,Chef and Owner Taïm New York NY
4,1,5,5,"""Yucca, Watermelon, Tortillas""",NaN,10-Feb-09,Geoffrey Zakarian,Alex Guarnaschelli,Marc Murphy,"watermelon, canned sardines, pepper jack chee...","beef shoulder, yucca, raisins, ancho chiles, ...","flour tortillas, prosecco, Canadian bacon, ro...",John Keller,Personal Chef New York NY,Andrea Bergquist,Executive Chef New York NY,Ed Witt,Executive Chef / Wine Director Bloomingdale ...,Josh Emett,Chef de cuisine Gordon Ramsay at The London ...


### Creating list of every appetizer ingredient

In [175]:
round_type = "entree"
round_list = chopped_data[round_type].to_list()
print(round_list[:5])

[' duck breast, green onions, ginger, honey ', ' daikon, pork loin, Napa cabbage, Thai chiles, Blue Point oysters ', ' ground beef, cannellini beans, tahini paste, grape jelly ', ' scallops, collard greens, anchovies, sour cream ', ' beef shoulder, yucca, raisins, ancho chiles, dill pickles ']


### Creating a modified sequence of data
One which decomposes a sentence so that a sentence like ["My name is Jordan"]  turns into ["My name", "My name is", "My name is Jordan"]

In [176]:
text = []

for selection in round_list:                     # For each basket selection in appetizers
    ingredients_comma =  selection.split(",")         # Convert each basket selection into a list of strings
    ingredients = []
    for ingredient in ingredients_comma:
        for element in ingredient.strip().split(" "):
            ingredients.append(element)
    for i in range(1, len(ingredients)):              # For as long as the list of strings is
        n_gram = ingredients[:i + 1]                  # create a fragment of the sentence
        text.append(n_gram)                           # and append it to text

print(text[:15])

[['duck', 'breast'], ['duck', 'breast', 'green'], ['duck', 'breast', 'green', 'onions'], ['duck', 'breast', 'green', 'onions', 'ginger'], ['duck', 'breast', 'green', 'onions', 'ginger', 'honey'], ['daikon', 'pork'], ['daikon', 'pork', 'loin'], ['daikon', 'pork', 'loin', 'Napa'], ['daikon', 'pork', 'loin', 'Napa', 'cabbage'], ['daikon', 'pork', 'loin', 'Napa', 'cabbage', 'Thai'], ['daikon', 'pork', 'loin', 'Napa', 'cabbage', 'Thai', 'chiles'], ['daikon', 'pork', 'loin', 'Napa', 'cabbage', 'Thai', 'chiles', 'Blue'], ['daikon', 'pork', 'loin', 'Napa', 'cabbage', 'Thai', 'chiles', 'Blue', 'Point'], ['daikon', 'pork', 'loin', 'Napa', 'cabbage', 'Thai', 'chiles', 'Blue', 'Point', 'oysters'], ['ground', 'beef']]


### Tokenizing our words

In [177]:
tokenizer = Tokenizer(num_words = 2000, oov_token = "<OOV>")    # Generate a tokenizer
tokenizer.fit_on_texts(text)                                    # and fit it on our text
sequences = tokenizer.texts_to_sequences(text)                  # Turn all of our text into texts
word_index = tokenizer.word_index                               
word_count = len(word_index) - 1

### Padding each element so that they are all the same length

In [178]:
max_len = max([len(sequence) for sequence in sequences])
sequences = np.array(pad_sequences(sequences, maxlen = max_len, padding = "pre"))
print(sequences)

[[   0    0    0 ...    0   17   19]
 [   0    0    0 ...   17   19   12]
 [   0    0    0 ...   19   12  136]
 ...
 [   0    0    0 ...   39  128 1313]
 [   0    0    0 ...  128 1313   69]
 [   0    0    0 ... 1313   69  353]]


### Creating our inputs and labels for our training data

In [179]:
xs = sequences[:,:-1]
labels = sequences[:,-1]

ys = tf.keras.utils.to_categorical(labels)

### Creating and compiling a model

In [180]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(word_count, 64, input_length = max_len - 1),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(20)),
    tf.keras.layers.Dense(word_count +2, activation = "softmax"),
])

model.compile(loss = "categorical_crossentropy", optimizer = "adam", metrics = ["accuracy"])

### Training our model

In [181]:
model.fit(xs, ys, epochs=300, verbose = 2)

Epoch 1/300
132/132 - 1s - loss: 7.1422 - accuracy: 0.0105
Epoch 2/300
132/132 - 1s - loss: 6.7576 - accuracy: 0.0126
Epoch 3/300
132/132 - 1s - loss: 6.6708 - accuracy: 0.0117
Epoch 4/300
132/132 - 1s - loss: 6.5834 - accuracy: 0.0121
Epoch 5/300
132/132 - 1s - loss: 6.5021 - accuracy: 0.0133
Epoch 6/300
132/132 - 1s - loss: 6.4231 - accuracy: 0.0148
Epoch 7/300
132/132 - 1s - loss: 6.3433 - accuracy: 0.0162
Epoch 8/300
132/132 - 1s - loss: 6.2524 - accuracy: 0.0183
Epoch 9/300
132/132 - 1s - loss: 6.1611 - accuracy: 0.0221
Epoch 10/300
132/132 - 1s - loss: 6.0635 - accuracy: 0.0243
Epoch 11/300
132/132 - 1s - loss: 5.9792 - accuracy: 0.0271
Epoch 12/300
132/132 - 1s - loss: 5.8957 - accuracy: 0.0276
Epoch 13/300
132/132 - 1s - loss: 5.8077 - accuracy: 0.0324
Epoch 14/300
132/132 - 1s - loss: 5.7227 - accuracy: 0.0366
Epoch 15/300
132/132 - 1s - loss: 5.6514 - accuracy: 0.0366
Epoch 16/300
132/132 - 1s - loss: 5.5759 - accuracy: 0.0390
Epoch 17/300
132/132 - 1s - loss: 5.4930 - accura

132/132 - 1s - loss: 1.0288 - accuracy: 0.7913
Epoch 138/300
132/132 - 1s - loss: 1.0005 - accuracy: 0.8006
Epoch 139/300
132/132 - 1s - loss: 0.9805 - accuracy: 0.8028
Epoch 140/300
132/132 - 1s - loss: 0.9597 - accuracy: 0.8037
Epoch 141/300
132/132 - 1s - loss: 0.9396 - accuracy: 0.8087
Epoch 142/300
132/132 - 1s - loss: 0.9298 - accuracy: 0.8144
Epoch 143/300
132/132 - 1s - loss: 0.9285 - accuracy: 0.8132
Epoch 144/300
132/132 - 1s - loss: 0.9707 - accuracy: 0.7985
Epoch 145/300
132/132 - 1s - loss: 0.9611 - accuracy: 0.7963
Epoch 146/300
132/132 - 1s - loss: 0.9136 - accuracy: 0.8125
Epoch 147/300
132/132 - 1s - loss: 0.8955 - accuracy: 0.8173
Epoch 148/300
132/132 - 1s - loss: 0.8789 - accuracy: 0.8206
Epoch 149/300
132/132 - 1s - loss: 0.8907 - accuracy: 0.8177
Epoch 150/300
132/132 - 1s - loss: 0.8597 - accuracy: 0.8182
Epoch 151/300
132/132 - 1s - loss: 0.8437 - accuracy: 0.8282
Epoch 152/300
132/132 - 1s - loss: 0.8358 - accuracy: 0.8296
Epoch 153/300
132/132 - 1s - loss: 0.8

Epoch 272/300
132/132 - 1s - loss: 0.2491 - accuracy: 0.9212
Epoch 273/300
132/132 - 1s - loss: 0.2485 - accuracy: 0.9232
Epoch 274/300
132/132 - 1s - loss: 0.2663 - accuracy: 0.9174
Epoch 275/300
132/132 - 1s - loss: 0.2693 - accuracy: 0.9191
Epoch 276/300
132/132 - 1s - loss: 0.2669 - accuracy: 0.9201
Epoch 277/300
132/132 - 1s - loss: 0.2486 - accuracy: 0.9227
Epoch 278/300
132/132 - 1s - loss: 0.2440 - accuracy: 0.9265
Epoch 279/300
132/132 - 1s - loss: 0.2875 - accuracy: 0.9143
Epoch 280/300
132/132 - 1s - loss: 0.2735 - accuracy: 0.9184
Epoch 281/300
132/132 - 1s - loss: 0.2497 - accuracy: 0.9220
Epoch 282/300
132/132 - 1s - loss: 0.2420 - accuracy: 0.9227
Epoch 283/300
132/132 - 1s - loss: 0.2357 - accuracy: 0.9262
Epoch 284/300
132/132 - 1s - loss: 0.2335 - accuracy: 0.9220
Epoch 285/300
132/132 - 1s - loss: 0.2291 - accuracy: 0.9227
Epoch 286/300
132/132 - 1s - loss: 0.2349 - accuracy: 0.9236
Epoch 287/300
132/132 - 1s - loss: 0.2298 - accuracy: 0.9220
Epoch 288/300
132/132 - 

### Making Predictions with our model

In [183]:
seed_ingredients = "chicken"
num_words = 6

for _ in range(num_words):
    token_list = tokenizer.texts_to_sequences([seed_ingredients])[0]
    token_list = pad_sequences([token_list], maxlen = max_len - 1, padding = "pre")
    predicted = model.predict_classes(token_list, verbose = 2)
    output_word = ""
    for word, i in tokenizer.word_index.items():
        if i == predicted:
            output_word = word
            break
    seed_ingredients += f" {output_word}"

print(seed_ingredients)

1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
chicken breast dandelion greens starfruit cherry liqueur
